In [ ]:
import os
import json
import pandas as pd
import openai
import difflib

openai.api_key = "REPLACE_ME"

In [ ]:
file_path = "Custom Target Market Analysis.xlsx"
sheet_name = "Framework"
df = pd.read_excel(file_path, sheet_name, engine="openpyxl")

In [ ]:
# Sometimes we mispell so we don't want to rely on a 1:1 ratio.
# this allows some margin for error like capitals and typos
def ordered_similarity_ratio(input_string, string_list):
    similarity_ratios = []
    for string in string_list:
        similarity_ratio = difflib.SequenceMatcher(None, input_string, string).ratio()
        similarity_ratios.append(similarity_ratio)

    return similarity_ratios

# Extract Framework

In [ ]:
target_market_attractiveness = {}

for i in range (10):
    factor = df[df.columns[0]].values[6+i]
    if type(factor) == str:
        target_market_attractiveness[factor] = {}

within_factor_levels = False
factor_level_desc = {}

for factor, lvl in zip(df[df.columns[0]].values[17:], df[df.columns[1]].values[17:]):
    
    if within_factor_levels:
        # Found a nan which means we have found all the level descriptions
        if type(factor) != str:
            # The factor will be the one with the id of the max value
            target_market_attractiveness[list(target_market_attractiveness.keys())[similarity.idxmax()]] = factor_level_desc
            
            # reinitialize the dictionary
            within_factor_levels = False
            factor_level_desc = {}
        
        else:
            factor_level_desc[factor] = lvl
        
    else:
        if type(factor) == str:
            similarity = pd.Series(ordered_similarity_ratio(factor, target_market_attractiveness.keys()))
            # We are above the threshold which means that the factor has been found
            if similarity.max() > 0.7:
                within_factor_levels = True

# Update last populated factor level range
if factor_level_desc:
    # The factor will be the one with the id of the max value
    target_market_attractiveness[list(target_market_attractiveness.keys())[similarity.idxmax()]] = factor_level_desc

print("TARGET MARKET ATTRACTIVENESS\n")
for factor in target_market_attractiveness:
    print(factor)
    for factor_lvl_desc,factor_lvl in target_market_attractiveness[factor].items():
        print(f"   {factor_lvl} - {factor_lvl_desc}")
    print("")

In [ ]:
competitive_advantage = {}

for i in range (10):
    factor = df[df.columns[3]].values[6+i]
    if type(factor) == str:
        competitive_advantage[factor] = {}

within_factor_levels = False
factor_level_desc = {}

for factor, lvl in zip(df[df.columns[3]].values[17:], df[df.columns[4]].values[17:]):
    
    if within_factor_levels:
        # Found a nan which means we have found all the level descriptions
        if type(factor) != str:
            # The factor will be the one with the id of the max value
            competitive_advantage[list(competitive_advantage.keys())[similarity.idxmax()]] = factor_level_desc
            
            # reinitialize the dictionary
            within_factor_levels = False
            factor_level_desc = {}
        
        else:
            factor_level_desc[factor] = lvl
        
    else:
        if type(factor) == str:
            similarity = pd.Series(ordered_similarity_ratio(factor, competitive_advantage.keys()))
            # We are above the threshold which means that the factor has been found
            if similarity.max() > 0.7:
                within_factor_levels = True

# Update last populated factor level range
if factor_level_desc:
    # The factor will be the one with the id of the max value
    competitive_advantage[list(competitive_advantage.keys())[similarity.idxmax()]] = factor_level_desc

print("COMPETITIVE ADVANTAGE\n")
for factor in competitive_advantage:
    print(factor)
    for factor_lvl_desc,factor_lvl in competitive_advantage[factor].items():
        print(f"   {factor_lvl} - {factor_lvl_desc}")
    print("")

# Prompt

In [ ]:
TARGET_MARKET = ["TargetMarket1"]

In [ ]:
prompt = f"We want to assess the attractiveness of the following target market: '{TARGET_MARKET[0]}'\n\n"
prompt += "The factors that we will access this market are presented below: \n\n"
for factor in target_market_attractiveness:
    prompt += f"When assessing {factor}, these are the rules:\n"
    for factor_lvl_desc,factor_lvl in target_market_attractiveness[factor].items():
        prompt += f"We attribute a value of {factor_lvl} if {factor_lvl_desc}\n"
    prompt += "\n"
    
prompt += f"Given this information, can you return a level for each of the factors that is representative of {TARGET_MARKET[0]} target market\n"
prompt += "Please return it in a json dictionary format with the factor and level only. Do not add any other text apart from that. Indent the json with 4 spaces."

print(prompt)

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {
            "role": "system", 
             "content": 
                """
            You are an outstanding financial analyst and were given the task to perform market research on a possible market segment.
            The company succces relies on your accuracy to categorize a segment and classify according to the factors and levels specified.
                """
        },
        {
            "role": "user", 
             "content": prompt
        },
    ]
)
print(response.choices[0].message.content)

In [ ]:
prompt = f"We want to assess our competitive advantage based in relation with factors where we are have an advantage. Can you classify those for the following target market: '{TARGET_MARKET[0]}'\n\n"
prompt += "The factors that we will access this market are presented below: \n\n"
for factor in competitive_advantage:
    prompt += f"When assessing {factor}, these are the rules:\n"
    for factor_lvl_desc,factor_lvl in competitive_advantage[factor].items():
        prompt += f"We attribute a value of {factor_lvl} if {factor_lvl_desc}\n"
    prompt += "\n"
    
prompt += f"Given this information, can you return a level for each of the factors that is our competitive advantage from a viewpoint of {TARGET_MARKET[0]} target market.\n"
prompt += "Please return it in a json dictionary format with the factor and level only. Do not add any other text apart from that. Indent the json with 4 spaces."

print(prompt)

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {
            "role": "system", 
             "content": 
                """
            You are an outstanding financial analyst and were given the task to perform market research on a possible market segment.
            The company succces relies on your accuracy to categorize a segment and classify according to the factors and levels specified.
                """
        },
        {
            "role": "user", 
             "content": prompt
        },
    ]
)
print(response.choices[0].message.content)

In [ ]:
market_segment = "TargetMarket1"

response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
            {
                "role": "system", 
                 "content": 
                    """
                You are an outstanding financial analyst and were given the task to perform market research on a possible market segment.
                The company succces relies on your description of the provided market segment and how they perform investment research.
                    """
            },
            {
                "role": "user", 
                 "content":
                    f"""
                Can you explain in detail how {market_segment} performs investment research.
                What are the individuals involved in the investment research process?
                What roles do they have, how big is the team, and what does their day to day look like.
                What tools are these people likely using?
                Go in as much detail as possible 
                    """
            },
        ]
    )

print(response.choices[0].message.content)

# Run for all Target Markets of interest

In [ ]:
TARGET_MARKET = [
    "TargetMarket1",
    "TargetMarket2",
    "TargetMarket3",
]

In [ ]:
for target_market in TARGET_MARKET:
    
    print(f"--------------- TARGET MARKET: {target_market} ---------------")
    
    print(f"\n--- {target_market} ATTRACTIVENESS ---\n")

    prompt = f"We want to assess the attractiveness of the following target market: '{target_market}'\n\n"
    prompt += "The factors that we will access this market are presented below: \n\n"
    for factor in target_market_attractiveness:
        prompt += f"When assessing {factor}, these are the rules:\n"
        for factor_lvl_desc,factor_lvl in target_market_attractiveness[factor].items():
            prompt += f"We attribute a value of {factor_lvl} if {factor_lvl_desc}\n"
        prompt += "\n"
    prompt += f"Given this information, can you return a level for each of the factors that is representative of {target_market} target market\n"
    prompt += "Please return it in a json dictionary format with the factor and level only. Indent the json with 4 spaces. Do not add any other text apart from that. Just the dictionary!"

    response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
            {
                "role": "system", 
                 "content": 
                    """
                You are an outstanding financial analyst and were given the task to perform market research on a possible market segment.
                The company succces relies on your accuracy to categorize a segment and classify according to the factors and levels specified.
                    """
            },
            {
                "role": "user", 
                 "content": prompt
            },
        ]
    )
    
    print(response.choices[0].message.content)
    
    print(f"\n--- {target_market} COMPETITIVE ADVANTAGE ---\n")

    prompt = f"We want to assess our competitive advantage based in relation with factors where we are have an advantage. Can you classify those for the following target market: '{target_market}'\n\n"
    prompt += "The factors that we will access this market are presented below: \n\n"
    for factor in competitive_advantage:
        prompt += f"When assessing {factor}, these are the rules:\n"
        for factor_lvl_desc,factor_lvl in competitive_advantage[factor].items():
            prompt += f"We attribute a value of {factor_lvl} if {factor_lvl_desc}\n"
        prompt += "\n"

    prompt += f"Given this information, can you return a level for each of the factors that is our competitive advantage from a viewpoint of {target_market} target market.\n"
    prompt += "Please return it in a json dictionary format with the factor and level only. Indent the json with 4 spaces. Do not add any other text apart from that. Just the dictionary!"

    response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
            {
                "role": "system", 
                 "content": 
                    """
                You are an outstanding financial analyst and were given the task to perform market research on a possible market segment.
                The company succces relies on your accuracy to categorize a segment and classify according to the factors and levels specified.
                    """
            },
            {
                "role": "user", 
                 "content": prompt
            },
        ]
    )
    
    print(response.choices[0].message.content)
    
    
    print(f"\n--- {target_market} DESCRIPTION ---\n")
    
    response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
            {
                "role": "system", 
                 "content": 
                    """
                You are an outstanding financial analyst and were given the task to perform market research on a possible market segment.
                The company succces relies on your description of the provided market segment and how they perform investment research.
                    """
            },
            {
                "role": "user", 
                 "content":
                    f"""
                Can you explain in detail how {target_market} performs investment research.
                What are the individuals involved in the investment research process?
                What roles do they have, how big is the team, and what does their day to day look like.
                What tools are these people likely using?
                Go in as much detail as possible 
                    """
            },
        ]
    )
    
    print(response.choices[0].message.content)
    
    print("\n\n")